In [2]:
#!/usr/bin/env python3
from pwn import *
from Crypto.Util.number import long_to_bytes
from Crypto.Util.Padding import unpad
import time

# Funzione per interagire con l'opzione "Generate your randoms"
def query_seed(r, s):
    r.sendlineafter("> ", "2")
    r.sendlineafter("What is your seed? ", str(s))
    line = r.recvline().strip()
    return int(line.decode(), 16)

# Funzione per ottenere l'encrypted flag (opzione 1)
def get_encrypted_flag(r):
    r.sendlineafter("> ", "1")
    enc_hex = r.recvline().strip().decode()
    return bytes.fromhex(enc_hex)

# Connettiamoci al server
# Modifica host e port con quelli corretti
r = remote("crng.challs.cyberchallenge.it", 9065)

# Leggiamo la stampa dei parametri generali
r.recvuntil("Generated parameters: (")
params = r.recvline().strip().decode()  # formato: "p, base)"
params = params.strip(")")
p_str, base_str = params.split(", ")
p = int(p_str)
base = int(base_str)
log.info(f"p = {p}")
log.info(f"base = {base}")

# Ricostruiamo secret_vector
secret_vector = [None] * 128

# STEP 1: Ricaviamo secret_vector[0] con seed = 2^127
seed0 = 1 << 127
out0 = query_seed(r, seed0)
# out0 = base^(1 + secret_vector[0]) mod p, con esponente compreso in [1, 257]
for e in range(1, 258):
    if pow(base, e, p) == out0:
        secret_vector[0] = e - 1
        log.info(f"secret_vector[0] = {secret_vector[0]}")
        break
assert secret_vector[0] is not None

# STEP 2: Per ogni i=1..127, usiamo seed = 2^127 + 2^(127-i)
for i in range(1, 128):
    seed_i = (1 << 127) + (1 << (127 - i))
    out_i = query_seed(r, seed_i)
    # out_i = base^(1 + secret_vector[0] + secret_vector[i]) mod p.
    # Ricostruiamo secret_vector[i] cercando e in [1, 513]
    found = False
    for e in range(1, 514):
        if pow(base, e, p) == out_i:
            secret_vector[i] = e - (1 + secret_vector[0])
            log.info(f"secret_vector[{i}] = {secret_vector[i]}")
            found = True
            break
    assert found, f"Non trovata secret_vector[{i}]"

# STEP 3: Otteniamo il flag cifrato
enc_flag = get_encrypted_flag(r)
log.info(f"Encrypted flag: {enc_flag.hex()}")

# Ora dobbiamo "indovinare" il seed usato per cifrare il flag.
# Poiché viene usato int(time.time()), ipotizziamo che il timestamp sia vicino all'attuale.
current_time = int(time.time())
flag = None

# La lunghezza dell'OTP è pari a quella del ciphertext (multiplo di 16)
blocks = len(enc_flag) // 16

# Funzione per simulare l'RNG con un dato seed e secret_vector già noti
def simulate_rng(seed, blocks):
    cnt = 0
    otp = b""
    for _ in range(blocks):
        x = seed + cnt
        bits = bin(x)[2:].rjust(128, '1')[-128:]
        cnt += 1
        exp = 1
        for j in range(128):
            if bits[j] == '1':
                exp += secret_vector[j]
        val = pow(base, exp, p)
        # Il metodo originale: long_to_bytes(val % p).rjust(16, b'\x00')[:16]
        otp += long_to_bytes(val % p).rjust(16, b'\x00')[:16]
    return otp

# Bruteforziamo il seed in un intervallo ragionevole (±300 secondi)
for candidate in range(current_time - 300, current_time + 300):
    otp_candidate = simulate_rng(candidate, blocks)
    plain = bytes([a ^ b for a, b in zip(enc_flag, otp_candidate)])
    try:
        pt = unpad(plain, 16)
    except Exception:
        continue
    if pt.startswith(b"CCIT{"):
        flag = pt.decode()
        log.success(f"Flag trovata: {flag}")
        break

if not flag:
    log.failure("Flag non trovata")

r.close()


[x] Opening connection to crng.challs.cyberchallenge.it on port 9065
[x] Opening connection to crng.challs.cyberchallenge.it on port 9065: Trying 5.75.221.48
[+] Opening connection to crng.challs.cyberchallenge.it on port 9065: Done
[*] p = 328720964026880930316856142479576294201
[*] base = 181978863805370720131983539001280506581


/var/folders/3n/vf15kv6n675_m6mzk8p8rfxr0000gn/T/ipykernel_99526/2004740074.py:25: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.recvuntil("Generated parameters: (")
/var/folders/3n/vf15kv6n675_m6mzk8p8rfxr0000gn/T/ipykernel_99526/2004740074.py:9: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.sendlineafter("> ", "2")
/Users/robertovicario/anaconda3/lib/python3.11/site-packages/pwnlib/tubes/tube.py:876: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  res = self.recvuntil(delim, timeout=timeout)
/var/folders/3n/vf15kv6n675_m6mzk8p8rfxr0000gn/T/ipykernel_99526/2004740074.py:10: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.sendlineafter("What is your seed? ", str(s))


[*] secret_vector[0] = 135
[*] secret_vector[1] = 57
[*] secret_vector[2] = 3
[*] secret_vector[3] = 7
[*] secret_vector[4] = 94
[*] secret_vector[5] = 76
[*] secret_vector[6] = 209
[*] secret_vector[7] = 16
[*] secret_vector[8] = 152
[*] secret_vector[9] = 155
[*] secret_vector[10] = 198
[*] secret_vector[11] = 189
[*] secret_vector[12] = 59
[*] secret_vector[13] = 201
[*] secret_vector[14] = 134
[*] secret_vector[15] = 53
[*] secret_vector[16] = 14
[*] secret_vector[17] = 113
[*] secret_vector[18] = 65
[*] secret_vector[19] = 147
[*] secret_vector[20] = 120
[*] secret_vector[21] = 181
[*] secret_vector[22] = 196
[*] secret_vector[23] = 72
[*] secret_vector[24] = 158
[*] secret_vector[25] = 123
[*] secret_vector[26] = 24
[*] secret_vector[27] = 240
[*] secret_vector[28] = 157
[*] secret_vector[29] = 215
[*] secret_vector[30] = 67
[*] secret_vector[31] = 190
[*] secret_vector[32] = 196
[*] secret_vector[33] = 14
[*] secret_vector[34] = 211
[*] secret_vector[35] = 65
[*] secret_vector[3

/var/folders/3n/vf15kv6n675_m6mzk8p8rfxr0000gn/T/ipykernel_99526/2004740074.py:16: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  r.sendlineafter("> ", "1")
